# Task 1


In [ ]:
import os
import glob
import shutil
import pandas as pd

In [ ]:
import os
import glob
import shutil
import pandas as pd

csv_files = glob.glob('csv_files/*.csv')
# Move each CSV file to "backup_folder"
for file in csv_files:
  shutil.move(file, 'backup_folder/')
  print(f"Moved File: {file}")


def export_data(df, file_name, format):
  if format == 'csv':
    df.to_csv(file_name, index=False)
    print(f"Data exported to {file_name} in CSV format.")
  elif format == 'json':
    df.to_json(file_name, orient="records")
    print(f"Data exported to {file_name} in JSON format.")
  else:
    print("Unsupported format.")

#sample data
data = {'Name': ['Alice', 'Alex', 'Bob'],
        'Age': [25, 22, 35],
        'City': ['New York', 'Los Angelos', 'Chicago']}
df = pd.DataFrame(data)

# Export DataFrame to CSV and JSON
export_data(df, "output.csv", "csv")
export_data(df, "output.json", "json")

Moved File: csv_files/iris.csv
Moved File: csv_files/tips.csv
Moved File: csv_files/banklist.csv
Data exported to output.csv in CSV format.
Data exported to output.json in JSON format.


# Task 2


In [ ]:
import yfinance as yf
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

db_name = "stocks.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Creating a table to store stock data
cursor.execute('''
CREATE TABLE IF NOT EXISTS stock_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    symbol TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER
)
''')

conn.commit()
print("Database and table created.")

Database and table created.


In [ ]:
import time

def fetch_stock_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        data = stock.history(period="1d", interval="1h")

        if data.empty:
            print(f"No data found for {symbol}. Skipping...")
            return None

        latest = data.iloc[-1]  # Get the most recent price data
        return {
            "symbol": symbol,
            "open": latest["Open"],
            "high": latest["High"],
            "low": latest["Low"],
            "close": latest["Close"],
            "volume": latest["Volume"]
        }
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None


def store_data(symbol):
    stock_data = fetch_stock_data(symbol)
    if stock_data:
        cursor.execute('''INSERT INTO stock_data (symbol, open, high, low, close, volume)
                          VALUES (?, ?, ?, ?, ?, ?)''',
                       (stock_data["symbol"], stock_data["open"], stock_data["high"],
                        stock_data["low"], stock_data["close"], stock_data["volume"]))
        conn.commit()
        print(f"Stored data for {symbol}")


def analyze_stock(symbol):
    df = pd.read_sql_query("SELECT * FROM stock_data WHERE symbol=? ORDER BY timestamp DESC LIMIT 100", conn, params=(symbol,))
    print(df)


symbol = "MSFT"

for _ in range(5):
    store_data(symbol)
    time.sleep(60)

analyze_stock(symbol)

# Close database connection
conn.close()

Stored data for MSFT
Stored data for MSFT
Stored data for MSFT
Stored data for MSFT
Stored data for MSFT
   id symbol            timestamp        open        high         low  \
0  15   MSFT  2025-03-04 18:09:30  386.024994  386.024994  384.220001   
1  14   MSFT  2025-03-04 18:08:30  386.024994  386.024994  384.220001   
2  13   MSFT  2025-03-04 18:07:30  386.024994  386.024994  384.220001   
3  12   MSFT  2025-03-04 18:06:30  386.024994  386.024994  384.220001   
4  11   MSFT  2025-03-04 18:05:30  386.024994  386.024994  384.220001   
5  10   MSFT  2025-03-04 17:59:53  384.641205  384.641205  384.641205   
6   9   MSFT  2025-03-04 17:58:53  384.549988  384.549988  384.549988   
7   8   MSFT  2025-03-04 17:57:52  384.579987  384.579987  384.579987   
8   7   MSFT  2025-03-04 17:56:52  384.670013  384.670013  384.670013   
9   6   MSFT  2025-03-04 17:55:52  384.829987  384.829987  384.829987   

        close   volume  
0  386.019989  1161266  
1  385.704987  1127359  
2  385.170013  1

# Task 3

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL (replace with a valid HTML website)
URL = "http://books.toscrape.com/"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def get_books(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        print(f"Failed to retrieve data: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.find_all("article", class_="product_pod")
    book_list = []

    for book in books:
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").text
        stock = book.find("p", class_="instock availability").text.strip()
        book_list.append({"Title": title, "Price": price, "Availability": stock})

    return book_list

# Fetch book data
books_data = get_books(URL)

# Save data to CSV
df = pd.DataFrame(books_data)
df.to_csv("books.csv", index=False)

# Print output
print(df)
print("Data saved to books.csv")


                                                Title    Price Availability
0                                A Light in the Attic  Â£51.77     In stock
1                                  Tipping the Velvet  Â£53.74     In stock
2                                          Soumission  Â£50.10     In stock
3                                       Sharp Objects  Â£47.82     In stock
4               Sapiens: A Brief History of Humankind  Â£54.23     In stock
5                                     The Requiem Red  Â£22.65     In stock
6   The Dirty Little Secrets of Getting Your Dream...  Â£33.34     In stock
7   The Coming Woman: A Novel Based on the Life of...  Â£17.93     In stock
8   The Boys in the Boat: Nine Americans and Their...  Â£22.60     In stock
9                                     The Black Maria  Â£52.15     In stock
10     Starving Hearts (Triangular Trade Trilogy, #1)  Â£13.99     In stock
11                              Shakespeare's Sonnets  Â£20.66     In stock
12          